In [664]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from factor_analyzer import FactorAnalyzer
import warnings
from factor_analyzer.factor_analyzer import calculate_kmo

warnings.simplefilter(action='ignore', category=FutureWarning)

# user input
state_abbr = None
area_study = None
control = None
carnegie = None
predom_deg = None

#df references
mean_df = pd.DataFrame()
std_df = pd.DataFrame()
mean_fos_df = pd.DataFrame()
std_fos_df = pd.DataFrame() 

# other var
fos_numstu_pct = None

#keys

cost_keys = ['NPT4', 'DEBT_MDN', 'NUM4', 'PCTFLOAN',]
completion_keys = ['C150', 'WDRAW_ORIG_YR2_RT', 'RET_FT4', 'LOAN_COMP_RT']
outcomes_keys = ['WNE_NWNE_RT','MD_EARN_WNE_P6', 'ADJ_STD_EARN', 'GT_28K_P6']
behavior_keys = ['RPY_3YR_RT_SUPP', 'CDR3','WDRAW_ORIG_YR2_RT', 'PPLUS_PCT_LOW']

In [665]:
#statistical functions
def zcore_to_percentile(z_score):
    return .5 * (math.erf(z_score / 2 ** .5) + 1)

def spec_dev(temp_df):
    temp_df['EARN_STDEV_FOS'] = (temp_df['EARN']-temp_df['EARN'].mean())/temp_df['EARN'].std()
    temp_df['EARN_STDEV_FOS'] = pd.Series([round(x, 2) for x in temp_df['EARN_STDEV_FOS']], index = temp_df.index)
    return temp_df

#fos functions

#dataframe creation functions
def instantiate_state(df):
    xf = df.copy(deep=True)
    if state_abbr:
        if state_abbr[0] == '-ALL-':
            xf=xf
        else:
            xf = xf.loc[xf['STABBR'].isin(state_abbr)]       
    return xf

def instantiate_cont(xf):
    if control:
        if control[0] == '-ALL-':
            xf = xf
        else:
                xf = xf.loc[xf['CONTROL'].isin(control)]
    return xf
        
def instantiate_carn(xf):
    if carnegie:
        if carnegie[0] == '-ALL-':
            xf = xf
        else:
            xf = xf.loc[xf['CCBASIC'].isin(carnegie)]        
    return xf

def instantiate_deg(xf):
    if predom_deg:
        if predom_deg[0] == '-ALL-':
            xf = xf
        else:
            xf = xf.loc[xf['PREDDEG'].isin(predom_deg)]
    return xf


#fos function

def instantiate_fos(xf):
    global fos_numstu_pct
    if area_study:
        if area_study[0] == '-ALL-':
            xf = xf
        else:
                xf = xf.loc[xf['FOS_GEN'].isin(area_study)]
                
    fos_numstu_pct = xf['TOTAL_EMP'].sum()/fos_df['TOTAL_EMP'].sum()
    return xf

def calc_fos_impact(f_df, g_df):
    as_df = f_df.copy(deep=True)
    xf = g_df.copy(deep=True)
    
    av_earn = as_df['EARN'].mean()
    earn_zscore = (av_earn - mean_fos_df['EARN'])/std_fos_df['EARN']
    
    av_emprate = as_df['EMP_RT'].mean()
    emprate_zscore = (av_emprate - mean_fos_df['EMP_RT'])/std_fos_df['EMP_RT']
    size_share = as_df['TOTAL_EMP'].sum()/fos_df['TOTAL_EMP'].sum()
    print('Program Earning Percentile: ','{:.2%}'.format(zcore_to_percentile(earn_zscore)))
    print('Program Employment Percentile: ','{:.2%}'.format(zcore_to_percentile(emprate_zscore)))
    
    earn_change = earn_zscore*std_df['MD_EARN_WNE_P6']
    emp_change = emprate_zscore*std_df['COUNT_WNE_P6']
    xf['MD_EARN_WNE_P6'] = xf['MD_EARN_WNE_P6'] + earn_change
    xf['COUNT_WNE_P6'] = xf['COUNT_WNE_P6'] + emp_change
    return xf
    
#scoring functions
def factor_master(df):
    load_map = {}
    ls,kms = factor_analysis_cost(df)
    la,kma = factor_analysis_completion(df)
    lo,kmo = factor_analysis_outcomes(df)
    lb,kmb = factor_analysis_behavior(df)
    load_map['cost'] = ls
    load_map['completion'] = la
    load_map['outcomes'] = lo
    load_map['behavior'] = lb
    
    kmdata = [kms, kma, kmo, kmb]
    kmo_df = pd.DataFrame(columns=['Cost','Completion','Outcomes','Behavior'])
    kmo_df.loc[0] = kmdata
    kmo_df = kmo_df.rename(index={0:'KMO'})
    return (load_map, kmo_df)

def ranker(df, lm):
    cost_score = []
    completion_score = []
    outcomes_score = []
    behavior_score = []
    
    sl = [abs(x) for x in lm['cost']] 
    cost_weight = sl/sum(sl)
    for key in cost_keys:
        zscore = (df[key].mean()-mean_df[key])/std_df[key]
        cost_score.append(zscore)
    sscore = sum([a*b for a, b in zip(cost_weight,cost_score)])[0]
    cost_score.append(sscore)
    cost_score.append('{:.2%}'.format(zcore_to_percentile(sscore)))
        
    al = [abs(x) for x in lm['completion']] 
    completion_weight = al/sum(al)
    for key in completion_keys:
        zscore = (df[key].mean()-mean_df[key])/std_df[key]
        completion_score.append(zscore)
    ascore = sum([a*b for a, b in zip(completion_weight,completion_score)])[0]
    completion_score.append(ascore)
    completion_score.append('{:.2%}'.format(zcore_to_percentile(ascore)))
        
    ol = [abs(x) for x in lm['outcomes']] 
    outcomes_weight = al/sum(ol)
    for key in outcomes_keys:
        zscore = (df[key].mean()-mean_df[key])/std_df[key]
        outcomes_score.append(zscore)
    oscore = sum([a*b for a, b in zip(outcomes_weight,outcomes_score)])[0]
    outcomes_score.append(oscore)
    outcomes_score.append('{:.2%}'.format(zcore_to_percentile(oscore)))
        
    bl = [abs(x) for x in lm['behavior']] 
    behavior_weight = al/sum(bl)
    for key in outcomes_keys:
        zscore = (df[key].mean()-mean_df[key])/std_df[key]
        behavior_score.append(zscore)
    bscore = sum([a*b for a, b in zip(behavior_weight,behavior_score)])[0]
    behavior_score.append(bscore)
    behavior_score.append('{:.2%}'.format(zcore_to_percentile(bscore)))
    
    d ={
        'Cost': cost_score,
        'Completion': completion_score,
        'Outcomes': outcomes_score,
        'Behavior': behavior_score,
       }
    return pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))

def describe_df(start_df):
    num_sch = start_df.shape[0]
    numsch_pct = '{:.1%}'.format(start_df.shape[0]/df.shape[0])
    total_stu = int(start_df['TOTALG12'].sum()*fos_numstu_pct)
    pct_stu = '{:.1%}'.format(total_stu/df['TOTALG12'].sum())
    a_cost_att = '${:,.0f}'.format(start_df['COSTT4'].mean())
    a_defaultrt_3yr = '{:.1%}'.format(start_df['CDR3'].mean())
    a_medearn_6yr = '${:,.0f}'.format(start_df['MD_EARN_WNE_P6'].mean())
    
    des_df = pd.DataFrame(columns=['# Schools', '% of All Schools', 'Total # of Students', '% of All Students', 
                                   'Av. Cost', 'Av. 3YR DefaultRate', 'Av. Income 6YR'])
    des_df.loc[0] = [num_sch, numsch_pct, total_stu, pct_stu, a_cost_att, a_defaultrt_3yr, a_medearn_6yr]
    des_df = des_df.rename(index={0:''})
    
    return des_df
    

In [666]:
# factor analysis
def factor_analysis_cost(df):
    df = df.copy(deep=True)
    data_df = df[cost_keys]
    data_df = data_df.replace(0, np.nan)
    data_df = data_df.fillna(data_df.mean())
    kmo_all,kmo_model=calculate_kmo(data_df)
    fa = FactorAnalyzer(rotation=None)
    fa.set_params(n_factors=1)
    fa.fit(data_df)
    loadings = fa.loadings_
    return (loadings, round(kmo_model,2))

def factor_analysis_completion(df):
    data_df = df[completion_keys]
    data_df = data_df.fillna(data_df.mean())
    kmo_all,kmo_model=calculate_kmo(data_df)
    fa = FactorAnalyzer(rotation=None)
    fa.set_params(n_factors=1)
    fa.fit(data_df)
    loadings = fa.loadings_
    return (loadings, round(kmo_model,2))

def factor_analysis_outcomes(df):
    df = df.copy(deep=True)
    data_df = df[outcomes_keys]
    data_df = data_df.fillna(data_df.mean())
    kmo_all,kmo_model=calculate_kmo(data_df)
    fa = FactorAnalyzer(rotation=None)
    fa.set_params(n_factors=1)
    fa.fit(data_df)
    loadings = fa.loadings_
    return (loadings, round(kmo_model,2))

def factor_analysis_behavior(df):
    df = df.copy(deep=True)
    data_df = df[behavior_keys]
    data_df = data_df.fillna(df.mean())
    kmo_all,kmo_model=calculate_kmo(data_df)
    fa = FactorAnalyzer(rotation=None)
    fa.set_params(n_factors=1)
    fa.fit(data_df)
    loadings = fa.loadings_
    return (loadings, round(kmo_model,2))

In [667]:
# college scorecard data reading and cleaning
df = pd.read_csv('sch_data_cscurated.csv', low_memory=False, na_values=['#DIV/0!'])
df = df[df['MAIN']==1]
df = df.dropna(axis=0,thresh=int(len(df.columns)*0.85))
df = df.drop(df[df['STABBR'] == 'GU'].index)
df = df.drop(df[df['STABBR'] == 'VI'].index)
df = df.drop(df[df['STABBR'] == 'PR'].index)
df['TOTALG12'] = df['UG12MN']+df['G12MN']
df['TOTAL_SPEND'] = df['TOTALG12']*df['NPT4']
df['LOAN_COMP_RT'] = df['LOAN_YR4_N']/df['NOLOAN_YR4_N']
df['WNE_NWNE_RT'] = df['COUNT_WNE_P6']/(df['COUNT_WNE_P6']+df['COUNT_NWNE_P6'])
df['ADJ_STD_EARN'] = 1-(df['SD_EARN_WNE_P10']/df['MN_EARN_WNE_P10'])

# recode carnegie classification
df['CCBASIC'] = df['CCBASIC'].replace([-2, 0, 33], 100)
df['CCBASIC'] = df['CCBASIC'].replace(np.arange(1,10), 101)
df['CCBASIC'] = df['CCBASIC'].replace([10, 11, 12, 13, 14], 102)
df['CCBASIC'] = df['CCBASIC'].replace([21, 22, 23], 103)
df['CCBASIC'] = df['CCBASIC'].replace([18, 19, 20], 104)
df['CCBASIC'] = df['CCBASIC'].replace([15, 16, 17], 105)
df['CCBASIC'] = df['CCBASIC'].replace(np.arange(24,33), 106)
df['CCBASIC'] = df['CCBASIC']-100

# save std and mean df's
mean_df = df.mean(axis=0, skipna=True, numeric_only=True)
std_df = df.std(axis=0, skipna=True, numeric_only=True)

In [668]:
# fos data cleaning
fos_df = pd.read_csv('fos_data.csv', low_memory=False)
fos_df = fos_df.dropna()
fos_df = fos_df.rename(columns={'TOTAL_EMP':'TOTAL'})
fos_df['EARN'] = pd.Series([math.trunc(x) for x in fos_df['EARN']], index = fos_df.index)
fos_df['TOTAL'] = pd.Series([math.trunc(x) for x in fos_df['TOTAL']], index = fos_df.index)
fos_df['TOTAL_EMP'] = (fos_df['TOTAL']*fos_df['EMP_RT']).astype('int64')
fos_df['EMP_RT'] = pd.Series([round(x, 4) for x in fos_df['EMP_RT']], index = fos_df.index)
fos_df['EARN_STDEV'] = (fos_df['EARN']-fos_df['EARN'].mean())/fos_df['EARN'].std()
fos_df['EARN_STDEV'] = pd.Series([round(x, 2) for x in fos_df['EARN_STDEV']], index = fos_df.index)
fos_df = fos_df.groupby('FOS_GEN').apply(spec_dev)

mean_fos_df = fos_df.mean(axis=0, skipna=True, numeric_only=True)
std_fos_df = fos_df.std(axis=0, skipna=True, numeric_only=True)

# state dictionary
state_map = df.groupby('STABBR')['INSTNM'].apply(list).to_dict()
for st in state_map:
    state_map[st].sort()
    
# State list
states = df['STABBR'].unique()
states = np.sort(states)
states = np.insert(states, 0, '-ALL-')

# FOS dictionary
fosname_map = fos_df.groupby('FOS_GEN')['FOS_SPEC'].apply(list).to_dict()
for fs in fosname_map:
    fosname_map[fs].sort()

fosgenrry = np.array([k for k, v in fosname_map.items()])
fosgenlist = np.insert(fosgenrry, 0, '-ALL-')

In [669]:
# styling
layout_button = widgets.Layout(width='324px', height='40px', left='90px', top='20px')
out_button = widgets.Layout(width='100px', height='50px', left='10px')

layout_vbo = widgets.Layout(height='500px')
layout_hbo = widgets.Layout(width='600px')

layout = widgets.Layout(width='415px', height='40px')
layout_half = widgets.Layout(width='415px', height='75px')
layout1 = widgets.Layout(width='415px', height='100px')
layout2 = widgets.Layout(width='415px', height='120px')
layout3 = widgets.Layout(width='415px', height='140px')
layout4 = widgets.Layout(width='415px', height='170px')

form_layout = widgets.Layout(width='850px', display='flex',flex_flow='row', justify_content='space-between')

# Buttons widget list
reset_widget = widgets.Button(description='Reset Inputs', disabled=False,
                               button_style='Danger', icon='circle-o-notch')
runsim_widget = widgets.Button(description='Generate Terms',disabled=False,button_style='success',icon='cog')

runscore_w = widgets.Button(description='Calculate Score',disabled=False,
                          button_style='success',icon='cog',layout=layout_button)

# Outputs (tab 1) list
output_1 = widgets.Output()
output_2 = widgets.Output()
output_3 = widgets.Output()
output_4 = widgets.Output()

# Outputs (tab 2) list
output_A = widgets.Output()

In [670]:
# GUI widget list (CUSTOM VARS)
state_ch = widgets.SelectMultiple(options=states, description="States",layout=layout4)
field_ch = widgets.SelectMultiple(options=fosgenlist, description="Programs",layout=layout4)

control_ch = widgets.SelectMultiple(options=[('-ALL-', '-ALL-'), ('Public', 1), 
                                       ('Private not-for-profit', 2), ('Private for-profit', 3)],
                              description="Institute Type",layout=layout_half)
carn_ch = widgets.SelectMultiple(options=[('-ALL-', '-ALL-'), ('Not Accredited', 0), ('Associate Colleges', 1), 
                                          ('Special Focus 2-Year Schools', 2), ('Baccalaureate Colleges', 3), 
                                       ('Masters Colleges/Universities', 4), ('Doctoral Universities', 5), 
                                          ('Special Focus 4-Year Schools', 6)],
                             description="Classifications",layout=layout3)
preddeg_ch = widgets.SelectMultiple(options=[('-ALL-', '-ALL-'),('Certificate',1),
                                         ('Associates',2),('Bachelors',3),('Graduate',4),],
                             description="Main Degrees",layout=layout2)


# tab widgets
customvar_box = widgets.HBox([widgets.VBox([state_ch, field_ch, runscore_w],layout=layout_vbo), 
                              widgets.VBox([control_ch, carn_ch, preddeg_ch],layout=layout_vbo)],
                             layout=form_layout)

sim_tab = widgets.Tab(children=[customvar_box])
sim_tab.set_title(0, 'Custom Variables')

In [671]:
# school update widget
def update_school_options(b=None):
    output_1.clear_output(wait=False)
    output_2.clear_output(wait=False)
    output_3.clear_output(wait=False)
    output_4.clear_output(wait=False)
    output_A.clear_output(wait=False)
    school_drop.options = state_map[states_drop.value] 
    with output_1:
        display(school_box)
        
def show_fos_options(b=None):
    output2.clear_output(wait=False)
    with output_2:
        display(fosgen_box)

def update_fos_options(b=None):
    output_3.clear_output(wait=False)
    fosspec_drop.options = fosname_map[fosgen_drop.value]
    with output_3:
        display(fosspec_box)
        
def generate_terms(b=None):
    pass 

def runscore_func(b=None):
    global state_abbr, area_study, control, carnegie, predom_deg
    output_A.clear_output(wait=False)
    
    state_abbr = state_ch.value
    area_study = field_ch.value
    control = control_ch.value
    carnegie = carn_ch.value
    predom_deg = preddeg_ch.value
    totalsize = df.shape[0]
    
    as_df = instantiate_fos(fos_df)
    df_1 = instantiate_state(df)
    df_2 = instantiate_cont(df_1)
    df_3 = instantiate_carn(df_2)
    df_4 = instantiate_deg(df_3)
    start_df = calc_fos_impact(as_df, df_4)
    load_map, kmo_df = factor_master(df)
    score_df = ranker(start_df, load_map).round(3)
    score_df = score_df.rename(index={ 0: 'F1',1: 'F2',2: 'F3',
                                      3: 'F4',4: 'Score',5: 'Percentile'})
    with output_A:
        display(score_df)
        display(kmo_df)
        display(describe_df(start_df))

In [672]:
#reset or run clicks
reset_widget.on_click(update_school_options)
runsim_widget.on_click(generate_terms)

#confirm clicks

#score clicks
runscore_w.on_click(runscore_func)

#display
display(sim_tab)
display(output_4)
display(output_A)

Output()

Output()